1. 预处理训练集，将训练集的猫与狗分为训练集和测试集，分别存放到对应的目录中

创建目录

In [7]:
import os,shutil

data_set_dir = './data_set'

train_dir = './train'

validation_dir = './validation'

test_dir = './test'

train_cats_dir = os.path.join(train_dir,'cat')

train_dogs_dir = os.path.join(train_dir,'dog')

cats_validation_dir = os.path.join(validation_dir,'cat')

dogs_validation_dir = os.path.join(validation_dir,'dog')

# os.mkdir(train_dir)

# os.mkdir(test_dir)

# os.mkdir(validation_dir)

# os.mkdir(train_cats_dir)

# os.mkdir(train_dogs_dir)

# os.mkdir(cats_validation_dir)

# os.mkdir(dogs_validation_dir)


训练数据迁移  划分 前 10000个为训练集 后2500个为测试集

In [20]:
def data_set_build(type,target_dir):

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (8000)]

    for file_name in file_names :
        src = os.path.join(data_set_dir,file_name)
        dst = os.path.join(target_dir,file_name)
        shutil.copyfile(src,dst)

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (8000,10000)]

    for file_name in file_names :
       src = os.path.join(data_set_dir,file_name)
       dst = os.path.join(validation_dir,type)
       dst = os.path.join(dst,file_name)
       shutil.copyfile(src,dst)

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (10000,12500)]

    for file_name in file_names :
        src = os.path.join(data_set_dir,file_name)
        dst = os.path.join(test_dir,file_name)
        shutil.copyfile(src,dst)

data_set_build('cat',train_cats_dir)
data_set_build('dog',train_dogs_dir)

导包

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


2024-06-01 16:14:48.117980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


普通卷积模型

In [2]:
def cnn_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

基于Resnet网络架构构建网络模型

In [32]:

def resnet_block(inputs, filters, strides=1, activation='relu'):
    x = Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters=filters, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    if strides != 1 or inputs.shape[-1] != filters:
        shortcut = Conv2D(filters=filters, kernel_size=1, strides=strides, padding='same')(inputs)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    
    x = Add()([x, shortcut])
    x = Activation(activation)(x)
    return x

def resnet_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    
    x = resnet_block(x, filters=64, strides=1)
    x = resnet_block(x, filters=64, strides=1)
    x = resnet_block(x, filters=64, strides=1)
    
    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128, strides=1)
    x = resnet_block(x, filters=128, strides=1)
    x = resnet_block(x, filters=128, strides=1)
    
    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256, strides=1)
    x = resnet_block(x, filters=256, strides=1)
    x = resnet_block(x, filters=256, strides=1)
    x = resnet_block(x, filters=256, strides=1)
    x = resnet_block(x, filters=256, strides=1)
    
    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512, strides=1)
    x = resnet_block(x, filters=512, strides=1)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model




准备开始模型训练

In [3]:
#input_shape = (223, 223, 3)

input_shape = (64, 64, 3)
# num_classes = 2
# model = resnet_model(input_shape, num_classes)

model = cnn_model()

model.summary()

/Users/liyehui/Desktop/school/CNN_Dogs_And_Cats/lab/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,453,569 (9.36 MB)

 Trainable params: 2,453,569 (9.36 MB)

 Non-trainable params: 0 (0.00 B)

配置训练器

In [4]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


数据预处理

In [8]:
train_image = ImageDataGenerator(rescale=1./255)
test_image = ImageDataGenerator(rescale=1./255)

train_generator = train_image.flow_from_directory(train_dir,
                                             target_size=(64,64),
                                             batch_size=20,
                                             class_mode='binary')

validation_generator = test_image.flow_from_directory(validation_dir,
                                                 target_size=(64,64),
                                                 batch_size=20,
                                                  class_mode='binary')



Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


训练网络模型

In [9]:
history = model.fit(train_generator, batch_size=100, epochs=80, validation_data=validation_generator)

Epoch 1/80


/Users/liyehui/Desktop/school/CNN_Dogs_And_Cats/lab/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


800/800 ━━━━━━━━━━━━━━━━━━━━ 85s 104ms/step - accuracy: 0.6030 - loss: 0.6476 - val_accuracy: 0.7335 - val_loss: 0.5318
Epoch 2/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 87s 108ms/step - accuracy: 0.7667 - loss: 0.4916 - val_accuracy: 0.7560 - val_loss: 0.4994
Epoch 3/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 89s 111ms/step - accuracy: 0.7935 - loss: 0.4361 - val_accuracy: 0.7910 - val_loss: 0.4388
Epoch 4/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 85s 106ms/step - accuracy: 0.8186 - loss: 0.3909 - val_accuracy: 0.7915 - val_loss: 0.4567
Epoch 5/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 86s 107ms/step - accuracy: 0.8487 - loss: 0.3363 - val_accuracy: 0.8248 - val_loss: 0.3986
Epoch 6/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 96s 119ms/step - accuracy: 0.8848 - loss: 0.2690 - val_accuracy: 0.8220 - val_loss: 0.3926
Epoch 7/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 94s 117ms/step - accuracy: 0.9108 - loss: 0.2159 - val_accuracy: 0.8363 - val_loss: 0.3826
Epoch 8/80
800/800 ━━━━━━━━━━━━━━━━━━━━ 91s 113ms/step - accuracy: 0.9383 - loss: 0.1572 - val

KeyboardInterrupt: 